In [ ]:
import numpy as np
import pandas as pd
import lib.recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import lib.areamanager
import math
import timeit
import lib.geo_utils
from collections import defaultdict
from lib.constants import experiment_constants
import lib.geocat.cat_utils as cat_utils
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
%matplotlib inline

from lib.usg.UserBasedCF import UserBasedCF
from lib.usg.FriendBasedCF import FriendBasedCF
from lib.usg.PowerLaw import PowerLaw

from lib.metrics import precisionk, recallk, ildk,gck,prk

# poi_int_id=df_checkin_train['business_id'].drop_duplicates().reset_index(drop=True).to_dict()
# poi_num=len(poi_int_to_id)
# user_int_id=df_checkin_train.reset_index().user_id.drop_duplicates().reset_index(drop=True).to_dict()
# user_num=len(user_int_to_id)
# for i,j in poi_int_id.copy().items():
#     poi_int_id[j]=i
# for i,j in user_int_id.copy().items():
#     user_int_id[j]=i
# def user_id_to_int(user_id):
#     return user_int_id[user_id]
# def poi_id_to_int(user_id):
#     return poi_int_id[user_id]

CITY=experiment_constants.get_city()

In [ ]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})

df_checkin_train=pd.read_csv("../data/checkin/train/"+CITY+".csv",converters={'categories':string_to_array})


users_id=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
pois_id=df_checkin['business_id'].drop_duplicates().reset_index(drop=True)
user_num=len(users_id)
poi_num=len(pois_id)
# USG uses unique checkins cij = 1 or cij = 0
#df_checkin_train=df_checkin_train.drop_duplicates(subset=['user_id','business_id']).reset_index(drop=True)

df_checkin_train=df_checkin_train.set_index('user_id')
df_checkin_train.head()

In [ ]:
df_checkin_test=pd.read_csv("../data/checkin/test/"+CITY+".csv",converters={'categories':string_to_array})


In [ ]:
ground_truth = defaultdict(set)
for index,row in df_checkin_test[['user_id','business_id']].drop_duplicates().iterrows():
    ground_truth[row['user_id']].add(row['business_id'])

In [ ]:
df_user_friends=pd.read_csv("../data/user/"+CITY+".csv",converters={'friends':string_to_array})

In [ ]:
poi_coos = {}
for lid,latlon in df_checkin[['business_id','latitude','longitude']].reset_index(drop=True).drop_duplicates().set_index('business_id').iterrows():
    poi_coos[lid] = tuple(latlon)

In [ ]:
poi_cats = {}
for lid,row in df_checkin[['business_id','categories']].reset_index(drop=True).drop_duplicates(subset=['business_id']).set_index('business_id').iterrows():
    poi_cats[lid] = row.categories


In [ ]:

dict_alias_title,category_tree,dict_alias_depth=cat_utils.cat_structs()
undirected_category_tree=category_tree.to_undirected()

In [ ]:
training_matrix = np.zeros((user_num, poi_num))


In [ ]:
for user_id,poi_id in df_checkin_train.business_id.iteritems():
   # print(user_id,poi_id)
    training_matrix[user_id,poi_id]=1

In [ ]:
social_relations = defaultdict(list)
for user_id,friends in df_user_friends.set_index("user_id").friends.iteritems():
    social_relations[user_int_id[user_id]]=[user_int_id[uid] for uid in friends]

In [ ]:
top_k = 100
alpha = 0.1
beta = 0.1

U = UserBasedCF()
S = FriendBasedCF(eta=0.05)
G = PowerLaw()

U.pre_compute_rec_scores(training_matrix)
S.compute_friend_sim(social_relations, training_matrix)
G.fit_distance_distribution(training_matrix, poi_coos)

In [ ]:
result_out = open("../data/result/sigir11_top_" + str(top_k) + ".txt", 'w')

all_uids = list(range(user_num))
all_lids = list(range(poi_num))



In [ ]:
np.random.shuffle(all_uids)

In [ ]:
precision, recall = [], []

In [ ]:
def normalize(scores):
    max_score = max(scores)
    if not max_score == 0:
        scores = [s / max_score for s in scores]
    return scores


In [ ]:
ild, gc = [],[]

In [ ]:
# df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
# df_poi_neighbor=df_poi_neighbor.set_index("business_id")

In [ ]:
# poi_neighbors={}
# for lid,row in df_poi_neighbor.iterrows():
#     neighbors=row['neighbors']
#     poi_neighbors[lid]=neighbors

In [ ]:
#log_size=training_matrix[0,training_matrix[0,:].nonzero()[0]].sum()

In [ ]:
pr=[]

In [ ]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")
poi_neighbors={}
for lid,row in df_poi_neighbor.iterrows():
    neighbors=row['neighbors']
    poi_neighbors[lid]=neighbors
df_poi_neighbor=None

In [ ]:
for cnt, uid in enumerate(all_uids):
    if uid in ground_truth:
        U_scores = normalize([U.predict(uid, lid)
                              if training_matrix[uid, lid] == 0 else -1
                              for lid in all_lids])
        S_scores = normalize([S.predict(uid, lid)
                              if training_matrix[uid, lid] == 0 else -1
                              for lid in all_lids])
        G_scores = normalize([G.predict(uid, lid)
                              if training_matrix[uid, lid] == 0 else -1
                              for lid in all_lids])
        
        U_scores = np.array(U_scores)
        S_scores = np.array(S_scores)
        G_scores = np.array(G_scores)
        
        overall_scores = (1.0 - alpha - beta) * U_scores + alpha * S_scores + beta * G_scores
        
        predicted = list(reversed(overall_scores.argsort()))[:top_k]
        actual = ground_truth[uid]
        
        precision.append(precisionk(actual, predicted[:10]))
        recall.append(recallk(actual, predicted[:10]))

        pr.append(prk(training_matrix[uid],predicted[:10],poi_neighbors))
        
        ild.append(ildk(predicted[:10],poi_cats,undirected_category_tree))
        gc.append(gck(uid,training_matrix,poi_cats,predicted[:10]))
        print(cnt, uid, "pre@10:", np.mean(precision), "rec@10:", np.mean(recall),"ild@10:",np.mean(ild),"gc@10:",np.mean(gc),"pr@10:",np.mean(pr))
        
        result_out.write(str(uid)+',"'+str(predicted)+'"\n')
        
result_out.close()

In [ ]:
df_metrics=pd.DataFrame([ild,gc,pr]).T
df_metrics.columns=['ild','gc','pr']

In [ ]:
df_metrics.plot()

In [ ]:
df_metrics.corr()

In [ ]:
import itertools
for m1,m2 in itertools.combinations(df_metrics.columns,2):
    df_metrics=df_metrics.sort_values(m1)
    df_metrics=df_metrics.reset_index(drop=True)
    df_metrics[[m1,m2]].plot()
    df_metrics=df_metrics.sort_values(m2)
    df_metrics=df_metrics.reset_index(drop=True)
    df_metrics[[m1,m2]].plot()